In [1]:
import numpy as np
from sympy import *
import matplotlib.pyplot as plt
%matplotlib inline
from celmech.nbody_simulation_utilities import set_time_step,align_simulation
from celmech.nbody_simulation_utilities import get_simarchive_integration_results
import rebound as rb
from os.path import isfile
from sympy import init_printing
init_printing()

In [2]:
from celmech.secular import LaplaceLagrangeSystem
from celmech.poincare import Poincare

In [10]:
def eccMatrix(alpha, mass):
    mu1, mu2, mu3 = mass
    alpha12, alpha23 = alpha
    P1, P2, P3 = alpha12**(3/2), 1, 1/alpha23**(3/2)

    sim = rb.Simulation()
    sim.add(m=1.)
    sim.add(m=mu1, P=P1)
    sim.add(m=mu2, P=P2)
    sim.add(m=mu3, P=P3)
    ps = sim.particles
    ps[1].r = ps[1].a*(ps[1].m/3/ps[0].m)**(1/3)
    ps[2].r = ps[2].a*(ps[2].m/3/ps[0].m)**(1/3)

    sim.move_to_com()
    sim.integrator = "whfast"
    sim.dt = sim.particles[1].P/20
    sim.collision = "direct"
    align_simulation(sim)
    
    pvars = Poincare.from_Simulation(sim)
    alpha12 = pvars.particles[1].a/pvars.particles[2].a
    alpha23 = pvars.particles[2].a/pvars.particles[3].a
    alpha13 = pvars.particles[1].a/pvars.particles[3].a
    print(f"alpha = {alpha12, alpha23, alpha13}")

    lsys = LaplaceLagrangeSystem.from_Simulation(sim)  
    M = lsys.Neccentricity_matrix
    print(f"eccentricity matrix =")
    print(M)
    
    return M

In [14]:
def eccMatrix_diag(alpha, mass):
    m1, m2, m3 = mass
    m_tot = m1+m2+m3
    
    R = np.array([[np.sqrt(m1/m_tot), np.sqrt(m2/m_tot), np.sqrt(m3/m_tot)],
             [-np.sqrt(m3/(m1+m3)), 0, np.sqrt(m1/(m1+m3))],
             [np.sqrt(m1*m2/(m1+m3)/m_tot), -np.sqrt((m1+m3)/m_tot), np.sqrt(m2*m3/(m1+m3)/m_tot)]])
    M = eccMatrix(alpha, mass)
    
    vals, k = np.linalg.eigh(M)
    
    return vals

In [15]:
eccMatrix_diag((0.9,0.94), (1e-8, 1e-8, 1e-8))

alpha = (0.8999999556192644, 0.9399999525825911, 0.8459999156064426)
eccentricity matrix =
[[-1.35752791e-06  9.27220410e-07  3.32624966e-07]
 [ 9.27220410e-07 -3.45180680e-06  2.44200545e-06]
 [ 3.32624966e-07  2.44200545e-06 -2.78247383e-06]]


array([-5.63775391e-06, -1.89193560e-06, -6.21190338e-08])